In [1]:
#import important libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
#reading df
df = pd.read_excel(r'C:\Users\risha\OneDrive\Desktop\Bishal\Run.xlsx')

In [3]:
df

,Unnamed: 0,URL,Title,Content
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto..."
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...
...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...


In [4]:
#import nltk
#nltk.download('punkt')

In [5]:
#finding 'POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE'
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')
def text_prep(x: str) -> list:
    corp = str(x).lower() 
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
    tokens = word_tokenize(corp)
    words = [t for t in tokens if t not in stop_words]
    lemmatize = [lemma.lemmatize(w) for w in words]
    return lemmatize
preprocess_tag = [text_prep(i) for i in df['content']]
df["preprocess_txt"] = preprocess_tag
df['WORD COUNT'] = df['preprocess_txt'].map(lambda x: len(x))

file = open('negative-words.txt')
neg_words = file.read().split()

file = open('positive-words.txt')
pos_words = file.read().split()

num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))

df['POSITIVE SCORE'] = num_pos

num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))

df['NEGATIVE SCORE'] = num_neg

df['POLARITY SCORE'] = ((df['POSITIVE SCORE'] - df['NEGATIVE SCORE'] / (df['POSITIVE SCORE'] + df['NEGATIVE SCORE']) + 0.000001))

df['SUBJECTIVITY SCORE'] = ((df['POSITIVE SCORE'] + df['NEGATIVE SCORE'] / df['WORD COUNT'] + 0.000001))

df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626
...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098


In [6]:
#storing df['content'] in sentences variable
sentences = df['Content']

In [7]:
#counting number of sentences in each rows
from nltk.tokenize import sent_tokenize
number_of_sentences=[]
for i in sentences:
    number_of_sentences.append(len(sent_tokenize(i)))
print(number_of_sentences)

[25, 54, 93, 87, 80, 90, 24, 46, 63, 80, 61, 76, 68, 43, 90, 30, 84, 52, 66, 69, 24, 28, 77, 15, 57, 48, 62, 74, 73, 35, 33, 76, 56, 27, 94, 36, 77, 80, 85, 95, 79, 64, 45, 69, 36, 81, 104, 58, 82, 63, 83, 22, 9, 56, 41, 12, 52, 56, 29, 111, 44, 17, 49, 24, 28, 11, 56, 10, 60, 20, 51, 74, 69, 82, 62, 65, 16, 70, 77, 77, 46, 82, 79, 54, 23, 37, 73, 195, 83, 60, 9, 44, 120, 94, 47, 62, 52, 74, 38, 59, 11, 60, 35, 50, 38, 12, 31, 32, 31, 47, 52, 53, 55, 70, 74, 11, 70, 80, 16, 88, 64, 86, 75, 66, 64, 33, 65, 45, 30, 50, 70, 59, 41, 14, 14, 20, 7, 14, 34, 41, 33, 75, 35, 38, 37, 49, 62, 66, 29, 66]


In [8]:
#counting number of words in each rows
from nltk.tokenize import word_tokenize
number_of_words=[]
for i in sentences:
    number_of_words.append(len(word_tokenize(i)))
print(number_of_words)

[746, 1060, 1973, 2041, 2048, 2056, 1283, 908, 1471, 2021, 1597, 1884, 1776, 1050, 2067, 785, 1877, 1135, 1862, 986, 801, 703, 2018, 472, 1393, 1258, 1533, 2020, 1798, 669, 744, 1591, 1286, 641, 1587, 1054, 2012, 1642, 1917, 1832, 1979, 1491, 841, 1731, 821, 2416, 2397, 1308, 1428, 1843, 1870, 564, 1895, 1080, 823, 353, 1159, 1529, 802, 2179, 765, 420, 1284, 696, 440, 182, 950, 203, 1319, 593, 1151, 1977, 1910, 1670, 996, 1616, 527, 1801, 1161, 1288, 1313, 1658, 1216, 1732, 843, 1326, 1923, 3976, 1946, 1691, 188, 1153, 2032, 2026, 1195, 1992, 1094, 1350, 1172, 1464, 212, 1239, 726, 1215, 1206, 431, 688, 1187, 798, 1185, 1154, 1159, 1439, 1729, 2014, 914, 2146, 1421, 382, 2388, 1727, 1972, 2015, 2174, 2067, 950, 1798, 1123, 765, 1887, 1961, 1580, 1127, 418, 361, 440, 262, 285, 698, 1147, 1099, 2015, 675, 1249, 1026, 1013, 1764, 1296, 791, 1150]


In [9]:
#calculating average numbers of sentence
avg=[]
for i,e in zip(number_of_sentences,number_of_words):
    avg.append(round((e/i),2))

In [10]:
avg

[29.84,
 19.63,
 21.22,
 23.46,
 25.6,
 22.84,
 53.46,
 19.74,
 23.35,
 25.26,
 26.18,
 24.79,
 26.12,
 24.42,
 22.97,
 26.17,
 22.35,
 21.83,
 28.21,
 14.29,
 33.38,
 25.11,
 26.21,
 31.47,
 24.44,
 26.21,
 24.73,
 27.3,
 24.63,
 19.11,
 22.55,
 20.93,
 22.96,
 23.74,
 16.88,
 29.28,
 26.13,
 20.52,
 22.55,
 19.28,
 25.05,
 23.3,
 18.69,
 25.09,
 22.81,
 29.83,
 23.05,
 22.55,
 17.41,
 29.25,
 22.53,
 25.64,
 210.56,
 19.29,
 20.07,
 29.42,
 22.29,
 27.3,
 27.66,
 19.63,
 17.39,
 24.71,
 26.2,
 29.0,
 15.71,
 16.55,
 16.96,
 20.3,
 21.98,
 29.65,
 22.57,
 26.72,
 27.68,
 20.37,
 16.06,
 24.86,
 32.94,
 25.73,
 15.08,
 16.73,
 28.54,
 20.22,
 15.39,
 32.07,
 36.65,
 35.84,
 26.34,
 20.39,
 23.45,
 28.18,
 20.89,
 26.2,
 16.93,
 21.55,
 25.43,
 32.13,
 21.04,
 18.24,
 30.84,
 24.81,
 19.27,
 20.65,
 20.74,
 24.3,
 31.74,
 35.92,
 22.19,
 37.09,
 25.74,
 25.21,
 22.19,
 21.87,
 26.16,
 24.7,
 27.22,
 83.09,
 30.66,
 17.76,
 23.88,
 27.14,
 26.98,
 22.93,
 26.87,
 32.94,
 32.3,
 28.79,
 2

In [11]:
df['AVG SENTENCE LENGTH'] = avg

In [12]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60
...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28


In [14]:
#function for counting complex words
def com(word):
    word_count=[]
    w=[]
    compl=[]
    count=0
    le = len(word)
    i = word.split(' ')
    for x in i[:le]:
        word_count.append(count)
        count=0
        for y in x:
            if y in ["a","i","e","o","u","A","E","I","O","U"]:
                count = count + 1
    for w in word_count:
        if w>=3:
            compl.append(w)
    return len(compl)

In [15]:
#calculating complex words
sentence=[]
cw=[]
for i in range(0,150):
    sentence.append(df['Content'][i])
    cw.append(com(sentence[i]))

In [16]:
df['COMPLEX WORD COUNT']= cw

In [17]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84,246
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63,268
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22,631
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46,674
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60,608
...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67,263
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45,452
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64,363
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28,254


In [18]:
#avg count of complex words
avgc=[]
for i,e in zip(cw,number_of_words):
    avgc.append(round((e/i),2))

In [19]:
avgc

[3.03,
 3.96,
 3.13,
 3.03,
 3.37,
 3.12,
 3.92,
 4.0,
 3.34,
 3.47,
 4.01,
 4.08,
 5.19,
 3.72,
 3.97,
 4.27,
 3.98,
 5.56,
 4.15,
 5.39,
 4.26,
 3.86,
 3.55,
 3.45,
 4.0,
 3.19,
 3.44,
 3.47,
 3.86,
 4.18,
 3.76,
 4.52,
 4.11,
 3.86,
 4.27,
 4.52,
 3.19,
 4.86,
 3.36,
 4.7,
 4.35,
 4.45,
 4.47,
 4.35,
 4.86,
 4.23,
 4.38,
 3.42,
 4.35,
 4.4,
 4.15,
 3.38,
 4.42,
 3.52,
 3.47,
 3.53,
 4.07,
 3.88,
 4.36,
 4.8,
 3.96,
 4.77,
 4.44,
 3.53,
 4.49,
 3.57,
 5.72,
 4.83,
 3.22,
 3.95,
 4.62,
 3.8,
 5.03,
 4.94,
 5.16,
 5.33,
 4.62,
 3.86,
 4.88,
 6.57,
 5.07,
 4.12,
 4.69,
 4.85,
 4.46,
 4.08,
 4.09,
 4.71,
 4.2,
 3.19,
 4.82,
 4.67,
 4.87,
 4.06,
 4.44,
 3.91,
 4.48,
 4.56,
 4.16,
 4.09,
 5.17,
 4.62,
 4.43,
 3.81,
 4.96,
 3.29,
 5.64,
 3.93,
 7.9,
 4.1,
 4.12,
 3.89,
 4.32,
 4.69,
 4.0,
 7.62,
 4.1,
 5.82,
 3.86,
 3.98,
 5.91,
 4.48,
 4.23,
 3.74,
 4.06,
 4.38,
 4.01,
 4.14,
 3.98,
 4.28,
 3.95,
 4.32,
 3.09,
 3.45,
 6.22,
 4.78,
 4.94,
 4.13,
 3.51,
 4.25,
 4.97,
 3.81,
 4.33,
 3.66,
 4.

In [20]:
df['PERCENTAGE OF COMPLEX WORDS'] = avgc

In [21]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84,246,3.03
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63,268,3.96
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22,631,3.13
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46,674,3.03
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60,608,3.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67,263,3.85
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45,452,3.90
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64,363,3.57
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28,254,3.11


In [23]:
#calculating fog index
fog=[]
for i,e in zip(avg,avgc):
    fog.append(round((0.4*(e/i)),2))

In [24]:
fog

[0.04,
 0.08,
 0.06,
 0.05,
 0.05,
 0.05,
 0.03,
 0.08,
 0.06,
 0.05,
 0.06,
 0.07,
 0.08,
 0.06,
 0.07,
 0.07,
 0.07,
 0.1,
 0.06,
 0.15,
 0.05,
 0.06,
 0.05,
 0.04,
 0.07,
 0.05,
 0.06,
 0.05,
 0.06,
 0.09,
 0.07,
 0.09,
 0.07,
 0.07,
 0.1,
 0.06,
 0.05,
 0.09,
 0.06,
 0.1,
 0.07,
 0.08,
 0.1,
 0.07,
 0.09,
 0.06,
 0.08,
 0.06,
 0.1,
 0.06,
 0.07,
 0.05,
 0.01,
 0.07,
 0.07,
 0.05,
 0.07,
 0.06,
 0.06,
 0.1,
 0.09,
 0.08,
 0.07,
 0.05,
 0.11,
 0.09,
 0.13,
 0.1,
 0.06,
 0.05,
 0.08,
 0.06,
 0.07,
 0.1,
 0.13,
 0.09,
 0.06,
 0.06,
 0.13,
 0.16,
 0.07,
 0.08,
 0.12,
 0.06,
 0.05,
 0.05,
 0.06,
 0.09,
 0.07,
 0.05,
 0.09,
 0.07,
 0.12,
 0.08,
 0.07,
 0.05,
 0.09,
 0.1,
 0.05,
 0.07,
 0.11,
 0.09,
 0.09,
 0.06,
 0.06,
 0.04,
 0.1,
 0.04,
 0.12,
 0.07,
 0.07,
 0.07,
 0.07,
 0.08,
 0.06,
 0.04,
 0.05,
 0.13,
 0.06,
 0.06,
 0.09,
 0.08,
 0.06,
 0.05,
 0.05,
 0.06,
 0.06,
 0.07,
 0.06,
 0.05,
 0.06,
 0.06,
 0.04,
 0.05,
 0.1,
 0.09,
 0.05,
 0.08,
 0.07,
 0.06,
 0.06,
 0.06,
 0.09,
 0.04,
 0.

In [25]:
df['FOG INDEX'] = fog

In [26]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84,246,3.03,0.04
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63,268,3.96,0.08
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22,631,3.13,0.06
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46,674,3.03,0.05
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60,608,3.37,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67,263,3.85,0.07
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45,452,3.90,0.05
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64,363,3.57,0.07
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28,254,3.11,0.05


In [45]:
#calculating personal pronouns
def pronoun(sentence):
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(sentence)
    return len(pronouns)

In [46]:
sentence=[]
p=[]
for i in range(0,150):
    sentence.append(df['Content'][i])
    p.append(pronoun(sentence[i]))

In [48]:
df['PERSONAL PRONOUNS'] = p

In [49]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,PERSONAL PRONOUNS
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84,246,3.03,0.04,0
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63,268,3.96,0.08,0
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22,631,3.13,0.06,1
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46,674,3.03,0.05,0
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60,608,3.37,0.05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67,263,3.85,0.07,9
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45,452,3.90,0.05,2
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64,363,3.57,0.07,2
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28,254,3.11,0.05,0


In [126]:
#
def nofc(sentence):
    number_of_characters=[]
    alphabets=0
    for i in range(len(sentence)):
        if(sentence[i].isalpha()):
            alphabets = alphabets + 1
    return alphabets

In [127]:
#counting character
sentence=[]
chars=[]
for i in range(0,150):
    sentence.append(df['Content'][i])
    chars.append(nofc(sentence[i]))

In [128]:
chars

[3752,
 4773,
 9906,
 10126,
 10077,
 10181,
 6029,
 4064,
 7164,
 9361,
 7218,
 8412,
 7165,
 4676,
 9105,
 3559,
 8727,
 4792,
 8342,
 3987,
 3649,
 3351,
 9612,
 2298,
 6134,
 6016,
 7537,
 9418,
 8473,
 2910,
 3391,
 7001,
 5804,
 2996,
 6954,
 4648,
 10115,
 6815,
 9193,
 7882,
 8767,
 6492,
 3705,
 7556,
 3492,
 10574,
 10139,
 6554,
 6096,
 8249,
 8301,
 2735,
 8593,
 4950,
 3935,
 1692,
 5261,
 6968,
 3539,
 9382,
 3437,
 1874,
 5483,
 3353,
 1994,
 875,
 3809,
 854,
 6503,
 2671,
 5101,
 9212,
 8261,
 6939,
 4018,
 6429,
 2189,
 8128,
 4707,
 5103,
 5823,
 7370,
 4987,
 7502,
 3918,
 6344,
 8600,
 16071,
 8787,
 8203,
 771,
 4644,
 8956,
 8839,
 5308,
 9122,
 4794,
 5823,
 5423,
 6550,
 825,
 5251,
 3032,
 5842,
 5063,
 2111,
 2819,
 5452,
 3024,
 5348,
 5423,
 5353,
 6265,
 7517,
 8970,
 3675,
 9874,
 5704,
 1783,
 11016,
 7164,
 8385,
 8956,
 9917,
 9377,
 4180,
 8239,
 4816,
 3433,
 8645,
 9062,
 6989,
 5774,
 2026,
 1414,
 1826,
 1065,
 1239,
 3341,
 5069,
 4709,
 9126,
 3

In [132]:
#calculating average word length
avl=[]
for i,e in zip(chars,number_of_words):
    avl.append(round(((e/i)),2))

In [133]:
avl

[0.2,
 0.22,
 0.2,
 0.2,
 0.2,
 0.2,
 0.21,
 0.22,
 0.21,
 0.22,
 0.22,
 0.22,
 0.25,
 0.22,
 0.23,
 0.22,
 0.22,
 0.24,
 0.22,
 0.25,
 0.22,
 0.21,
 0.21,
 0.21,
 0.23,
 0.21,
 0.2,
 0.21,
 0.21,
 0.23,
 0.22,
 0.23,
 0.22,
 0.21,
 0.23,
 0.23,
 0.2,
 0.24,
 0.21,
 0.23,
 0.23,
 0.23,
 0.23,
 0.23,
 0.24,
 0.23,
 0.24,
 0.2,
 0.23,
 0.22,
 0.23,
 0.21,
 0.22,
 0.22,
 0.21,
 0.21,
 0.22,
 0.22,
 0.23,
 0.23,
 0.22,
 0.22,
 0.23,
 0.21,
 0.22,
 0.21,
 0.25,
 0.24,
 0.2,
 0.22,
 0.23,
 0.21,
 0.23,
 0.24,
 0.25,
 0.25,
 0.24,
 0.22,
 0.25,
 0.25,
 0.23,
 0.22,
 0.24,
 0.23,
 0.22,
 0.21,
 0.22,
 0.25,
 0.22,
 0.21,
 0.24,
 0.25,
 0.23,
 0.23,
 0.23,
 0.22,
 0.23,
 0.23,
 0.22,
 0.22,
 0.26,
 0.24,
 0.24,
 0.21,
 0.24,
 0.2,
 0.24,
 0.22,
 0.26,
 0.22,
 0.21,
 0.22,
 0.23,
 0.23,
 0.22,
 0.25,
 0.22,
 0.25,
 0.21,
 0.22,
 0.24,
 0.24,
 0.22,
 0.22,
 0.22,
 0.23,
 0.22,
 0.23,
 0.22,
 0.22,
 0.22,
 0.23,
 0.2,
 0.21,
 0.26,
 0.24,
 0.25,
 0.23,
 0.21,
 0.23,
 0.23,
 0.22,
 0.22,
 0.23,
 0.

In [134]:
df['AVG WORD LENGTH'] = avl  

In [135]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84,246,3.03,0.04,0,0.20
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63,268,3.96,0.08,0,0.22
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22,631,3.13,0.06,1,0.20
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46,674,3.03,0.05,0,0.20
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60,608,3.37,0.05,1,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67,263,3.85,0.07,9,0.21
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45,452,3.90,0.05,2,0.23
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64,363,3.57,0.07,2,0.22
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28,254,3.11,0.05,0,0.19


In [140]:
#function to find syllables 
def syw(word):
    word_count=[]
    compl=[]
    count=0
    le = len(word)
    i = word.split(' ')
    for x in i[:le]:
        word_count.append(count)
        count=0
        for y in x:
            if y in ["a","i","e","o","u","A","E","I","O","U"]:
                count = count + 1
                if word.endswith("es" and "ed"):
                    count= count - 1
        compl.append(count)
    return len(compl)

In [141]:
sentence=[]
sylw=[]
for i in range(0,150):
    sentence.append(df['Content'][i])
    sylw.append(syw(sentence[i]))

In [142]:
sylw

[691,
 961,
 1759,
 1779,
 1828,
 1800,
 1194,
 811,
 1318,
 1800,
 1422,
 1669,
 1539,
 916,
 1784,
 713,
 1677,
 1043,
 1669,
 853,
 714,
 634,
 1807,
 435,
 1186,
 1118,
 1379,
 1789,
 1586,
 591,
 640,
 1405,
 1212,
 574,
 1395,
 953,
 1794,
 1415,
 1703,
 1655,
 1745,
 1304,
 738,
 1496,
 728,
 2159,
 1981,
 1192,
 1233,
 1698,
 1643,
 501,
 1812,
 964,
 722,
 322,
 1025,
 1382,
 722,
 1962,
 686,
 377,
 1137,
 625,
 390,
 165,
 808,
 183,
 1162,
 530,
 1057,
 1723,
 1752,
 1454,
 887,
 1415,
 470,
 1594,
 984,
 1131,
 1197,
 1453,
 1036,
 1569,
 759,
 1186,
 1693,
 3457,
 1766,
 1507,
 165,
 947,
 1831,
 1754,
 1067,
 1773,
 982,
 1213,
 1101,
 1281,
 187,
 1103,
 631,
 1120,
 1079,
 386,
 614,
 1036,
 727,
 1043,
 1031,
 1014,
 1277,
 1563,
 1828,
 878,
 1954,
 1270,
 347,
 2145,
 1616,
 1718,
 1760,
 2150,
 1831,
 839,
 1581,
 988,
 714,
 1632,
 1751,
 1379,
 1002,
 360,
 322,
 395,
 214,
 254,
 633,
 1050,
 997,
 1822,
 623,
 1124,
 949,
 913,
 1568,
 1169,
 712,
 1049]

In [155]:
df['SYLLABLE PER WORD'] = sylw  

In [156]:
df

,Unnamed: 0,URL,Title,Content,preprocess_txt,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,PERSONAL PRONOUNS,AVG WORD LENGTH,SYLLABLE PER WORD
0,0,https://insights.blackcoffer.com/is-telehealth...,Is telehealth the future of healthcare?,Covid-19 has paved the way for advancements i...,"[covid, paved, way, advancement, telehealth, s...",408,49,11,48.816668,49.026962,29.84,246,3.03,0.04,0,0.20,691
1,1,https://insights.blackcoffer.com/how-telehealt...,How Telehealth and Telemedicine helping people...,The last 2 years are the most critical time i...,"[last, year, critical, time, world, survive, t...",487,69,28,68.711341,69.057496,19.63,268,3.96,0.08,0,0.22,961
2,2,https://insights.blackcoffer.com/is-telemedici...,Is telemedicine effective in treating patients?,"Telemedicine, which allows patients and docto...","[telemedicine, allows, patient, doctor, commun...",1072,142,54,141.724491,142.050374,21.22,631,3.13,0.06,1,0.20,1759
3,3,https://insights.blackcoffer.com/is-telehealth...,The Future of Telehealth Services,Telemedicine refers to a specific set of clin...,"[telemedicine, refers, specific, set, clinical...",1139,126,49,125.720001,126.043021,23.46,674,3.03,0.05,0,0.20,1779
4,4,https://insights.blackcoffer.com/how-people-di...,How are people diverted to Telehealth services...,The future of telehealth stays positive To as...,"[future, telehealth, stay, positive, assist, a...",1046,122,54,121.693183,122.051626,25.60,608,3.37,0.05,1,0.20,1828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of a...,"[reconciling, financial, reality, mba, educati...",563,38,37,37.506668,38.065720,20.67,263,3.85,0.07,9,0.21,913
146,146,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment? An investment is a res...,"[investment, investment, resource, thing, proc...",929,58,30,57.659092,58.032294,28.45,452,3.90,0.05,2,0.23,1568
147,147,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision...,"[quality, affordable, healthcare, vision, gove...",678,49,56,48.466668,49.082597,19.64,363,3.57,0.07,2,0.22,1169
148,148,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process...,"[analytics, statistical, scientific, process, ...",423,54,14,53.794119,54.033098,27.28,254,3.11,0.05,0,0.19,712


In [157]:
#saving the file in excel form
df.to_excel('Score.xlsx')

In [173]:
output = pd.dfFrame(df,columns=['URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])

In [175]:
output

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
1,https://insights.blackcoffer.com/is-telehealth...,49,11,48.816668,49.026962,29.84,3.03,0.04,NaN,246,408,691,0,0.20
2,https://insights.blackcoffer.com/how-telehealt...,69,28,68.711341,69.057496,19.63,3.96,0.08,NaN,268,487,961,0,0.22
3,https://insights.blackcoffer.com/is-telemedici...,142,54,141.724491,142.050374,21.22,3.13,0.06,NaN,631,1072,1759,1,0.20
4,https://insights.blackcoffer.com/is-telehealth...,126,49,125.720001,126.043021,23.46,3.03,0.05,NaN,674,1139,1779,0,0.20
5,https://insights.blackcoffer.com/how-people-di...,122,54,121.693183,122.051626,25.60,3.37,0.05,NaN,608,1046,1828,1,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,https://insights.blackcoffer.com/blockchain-fo...,38,37,37.506668,38.065720,20.67,3.85,0.07,NaN,263,563,913,9,0.21
147,https://insights.blackcoffer.com/the-future-of...,58,30,57.659092,58.032294,28.45,3.90,0.05,NaN,452,929,1568,2,0.23
148,https://insights.blackcoffer.com/big-data-anal...,49,56,48.466668,49.082597,19.64,3.57,0.07,NaN,363,678,1169,2,0.22
149,https://insights.blackcoffer.com/business-anal...,54,14,53.794119,54.033098,27.28,3.11,0.05,NaN,254,423,712,0,0.19
